In [1]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random

[nltk_data] Downloading package punkt to C:\Users\E-M-
[nltk_data]     W-E\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package wordnet to C:\Users\E-M-
[nltk_data]     W-E\AppData\Roaming\nltk_data...
[nltk_data] Downloading package omw-1.4 to C:\Users\E-M-
[nltk_data]     W-E\AppData\Roaming\nltk_data...


In [2]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [3]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # adding documents
        documents.append((w, intent['tag']))

        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [4]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")

print (len(classes), "classes", classes)

print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

5634 documents
415 classes ['2018-04-10', '2018-04-12', '2018-04-13', '2018-04-14', '2018-04-16', '2018-04-17', '2018-04-18', '2018-04-23', '2018-04-24', '2018-04-25', '2018-04-26', '2018-04-27', '2018-05-1', '2018-05-10', '2018-05-11', '2018-05-12', '2018-05-14', '2018-05-15', '2018-05-16', '2018-05-17', '2018-05-21', '2018-05-22', '2018-05-24', '2018-05-25', '2018-05-27', '2018-05-28', '2018-05-30', '2018-05-4', '2018-05-5', '2018-05-8', '2018-06-10', '2018-06-12', '2018-06-16', '2018-06-17', '2018-06-19', '2018-06-2', '2018-06-22', '2018-06-24', '2018-06-27', '2018-06-29', '2018-06-4', '2018-06-6', '2018-06-7', '2018-06-8', '2018-07-16', '2018-07-18', '2018-07-19', '2018-07-2', '2018-07-20', '2018-07-23', '2018-07-24', '2018-07-26', '2018-07-29', '2018-07-3', '2018-07-31', '2018-07-6', '2018-07-7', '2018-07-8', '2018-08-10', '2018-08-11', '2018-08-12', '2018-08-14', '2018-08-15', '2018-08-2', '2018-08-20', '2018-08-21', '2018-08-23', '2018-08-28', '2018-08-29', '2018-08-3', '2018-08

In [5]:
# initializing training data
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


C:\Users\E-M-W-E\AppData\Local\Temp\ipykernel_5736\74795701.py:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [ ]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

C:\Users\E-M-W-E\AppData\Roaming\Python\Python39\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/200
1127/1127 [==============================] - 13s 10ms/step - loss: 2.1869 - accuracy: 0.7416
Epoch 2/200
1127/1127 [==============================] - 11s 10ms/step - loss: 1.9511 - accuracy: 0.7448
Epoch 3/200
1127/1127 [==============================] - 11s 10ms/step - loss: 1.9033 - accuracy: 0.7448
Epoch 4/200
1127/1127 [==============================] - 11s 10ms/step - loss: 1.8594 - accuracy: 0.7455
Epoch 5/200
1127/1127 [==============================] - 11s 10ms/step - loss: 1.8150 - accuracy: 0.7453
Epoch 6/200
1127/1127 [==============================] - 11s 10ms/step - loss: 1.8010 - accuracy: 0.7460
Epoch 7/200
1127/1127 [==============================] - 11s 10ms/step - loss: 1.7820 - accuracy: 0.7451
Epoch 8/200
1127/1127 [==============================] - 11s 10ms/step - loss: 1.7417 - accuracy: 0.7474
Epoch 9/200
1127/1127 [==============================] - 11s 10ms/step - loss: 1.7350 - accuracy: 0.7478
Epoch 10/200
1127/1127 [==============================]

In [ ]:
from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [ ]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res